## Vector Geometry

* axis system and points in space
* vector - particularly
* transformation


In [4]:
class Point3d 
{
    double m_x = 0.0;
    double m_y = 0.0;
    double m_z = 0.0;
    
    public Point3d()
    {
    
    }
    public Point3d(double x, double y, double z)
    {
        m_x = x;
        m_y = y;
        m_z = z;
    }
    
    public double getX()
    {
        return m_x;
    }
    
    public double getY()
    {
        return m_y;
    }
    
    public double getZ()
    {
        return m_z;
    }
    
    public double distance(Point3d pt)
    {
        double dx = m_x - pt.getX();
        double dy = m_y - pt.getY();
        double dz = m_z - pt.getZ();
        return Math.sqrt((dx * dx) + (dy * dy) + (dz * dz));
    }
}

In [16]:
Point3d pt1 = new Point3d();
Point3d pt2 = new Point3d(-3.0,3.0,5.0);
pt1.distance(pt2);

6.557438524302

In [17]:
class Vector3d {
    double m_i = 1.0;
    double m_j = 0.0;
    double m_k = 0.0;
    
    public Vector3d()
    {
    
    }
    
    public Vector3d(double i, double j, double k)
    {
        m_i = i;
        m_j = j;
        m_k = k;
    }
    
    public Vector3d(Point3d start, Point3d end)
    {
        m_i = end.getX() - start.getX();
        m_j = end.getY() - start.getY();
        m_k = end.getZ() - start.getZ();
    }
    
    public double getI()
    {
        return m_i;
    }
    
    public double getJ()
    {
        return m_j;
    }
    
    public double getK()
    {
        return m_k;
    }
    
    public Vector3d normalize()
    {
        double length = Math.sqrt((m_i * m_i) + (m_j * m_j) + (m_k * m_k));
        m_i /= length;
        m_j /= length;
        m_k /= length;
        return this;
    }
}





In [18]:
Vector3d targetDir = new Vector3d(pt1, pt2);
System.out.println("i,j,k = " + targetDir.getI() + ", " + targetDir.getJ() + ", " + targetDir.getK());
targetDir.normalize();
System.out.println("i,j,k = " + targetDir.getI() + ", " + targetDir.getJ() + ", " + targetDir.getK());



i,j,k = -3.0, 3.0, 5.0
i,j,k = -0.457495710997814, 0.457495710997814, 0.7624928516630234
